In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import cm 
import numpy as np
import math
import random
from IPython.display import clear_output
import imageio
import os
import time

In [9]:
__author__      = "Alexander Guy"
__version__     = "1.3.7"
__email__       = "ag518@sussex.ac.uk"
__status__      = "Assignment Release"

In [10]:
# Font alteration for any plots of differing sizes and scales

def font_size(size):
    font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : size}
    plt.rc('font', **font)

In [11]:
# Landscape functions, these allow us to test how hardy the slime climb algorithm is when facing different types of landscapes
def NK_Landscape(x, y):
    return 4*(1-x)**2*np.exp(-(x**2)-(y+1)**2) -15*(x/5 - x**3 - y**5)*np.exp(-x**2-y**2) -(1./3)*np.exp(-(x+1)**2 - y**2)-1*(2*(x-3)**7 -0.3*(y-4)**5+(y-3)**9)*np.exp(-(x-3)**2-(y-3)**2)

def rastrigin(X,Y):
    Z = (X**2 - 10 * np.cos(2 * np.pi * X)) + \
    (Y**2 - 10 * np.cos(2 * np.pi * Y)) + 20
    return(Z)

# Slime Climb functions

**P_Roll** () Here we weight the stoachstic process of diciding what a given bifurication should be for the end of a tendril.
this is based on the bifurcation of the Physarum polycephalum network over time, to stop this bifurication rate from being
exactly predictable we instead make it a chance roll to see what will happen, though if the height change between two nodes
in a given tendril are large enough this will accentuate this bifurication so you get explosive tri-frok bifurications on peaks

In [12]:
def P_roll(x,x_prev,t_span,t):
    
    t = (x-x_prev)*t*2/t_span
    
    M_S_P = 7.5/t_span*25 # Midsection proportion for ensuring the midpoint of these probability distributions match slime molds actual behaviour
    P_F = 25/t_span       # Proportionality factor, for scaling the gradient of the sigmoid

    p0 =  0.5 / (1 + np.exp((0.4*t*P_F) - (M_S_P*0.4*P_F))) # P0 is a null state in the study used for the proportion of material that is yet to be part of the slime mold network 
    p1 = ((0.2) / (1 + np.exp((0.5*t*P_F) - (M_S_P*0.5*P_F)))) + 0.25 + (p0/2) # As P4 is initially 0 and p0 rapidly decays to zero we add the probability from p0 to p1 and p3 as to best represent the percent chance of specific 
    p3 = ((0.6) / (1 + np.exp((-1)*((0.4*t*P_F) - (M_S_P*0.4*P_F))))) + 0.04 + (p0/2) #number of branches for a given bifurication
    p4 = 1 - p1 - p3

    rand_list = np.ones(int(p1*100))
    rand_list = np.concatenate((rand_list, (np.multiply(np.ones(int(p3*100)),2))), axis=None)
    rand_list = np.concatenate((rand_list, (np.multiply(np.ones(int(p4*100)),3))), axis=None)
    rand_list = np.random.permutation(rand_list)

    rand_roll = random.randint(0,(len(rand_list)-1))

    return(int(rand_list[rand_roll]))

**angle_spread** () Similarly based on studies of the slime mold the angles of a bifurication or tendril are based on 'nutrition'we treat change in height as nutrition for the same effect so not only do the tendrils bifuricate more aggressively at optima they also fan out in an extremely broad manner to maximise change of finding most 'nutritious' spot
This angle is usually within a range between an upper and lower bounds, both of which are best represented as sigmoid functions
Whilst there was no definitive source on how the angle range changed over time, seeing as the bifurication emulated sigmoids
and we want to ensure the angles dont exceed say 360 degrees for a particularly 'nutritious' spot a sigmoid locks these upper bounds to an absolute limit

In [13]:
def angle_spread(x,x_prev,t_span,t):

    X_prop = (x-x_prev)*t

    angle_low  =  (45 / (1 + np.exp((-1)*((10*X_prop/t_span) - (0.5*10))))) 
    angle_high =  (80 / (1 + np.exp((-1)*((10*X_prop/t_span) - (0.5*10)))))    + 45

    return(angle_low,angle_high)

angle_spread(2,1,50,25)

(22.5, 85.0)

**dist_func** () The distance function is not a sigmoid, more a polynomial line which reduces as nutrition gets particularly high and increases in areas that include harmful chemicals like pesticides. This function creates and approximation of the sigmoid, once again we don't want to allow the distance to become negative or extremely large so we lock upper and lower limits with if loops to avoid some 200m sprint away from the search space

These three functions used in conjunction result in a tendril branch that, when there is nothing of interest (minimas) keeps it's angles tight and increases segment distance.
Whereas if a source of nutrition is found, the tendrils branch out, increase their angles to explore as much of the local area as possible and decrease distances as to refine its search.

In [14]:
def dist_func(x,x_prev):

    diff = (x-x_prev)

    dist_return = (-0.226*(diff**3) + 1.22*(diff**2) - 2.69*(diff) + 3.03)*0.5
    if dist_return > 0.7:
        dist_return = 0.7

    if dist_return < 0.2:
        dist_return = 0.2
        
    return(dist_return)

**chemotaxis** () Physarum polycephalum uses chemoreceptors to detect chemical signatures in the air and will move off with its tendrils in that direction. To approximate this we throw a very sparse net over its immediate surrounding area (as far as it can travel for the number of increments *(dist • t_span)*). For each point in this sparse meshgrid it retrives the highest points and their coordinates, find the n highest values and calculates and returns the starting angles for the slime mold to travel towards.

In [15]:
def chemotaxis(threshold,x,y,t_span,dist,land_func):

    radius = t_span*dist
    x_grid = np.linspace((x-radius),(x+radius),20)
    y_grid = np.linspace((y-radius),(y+radius),20)
    xx, yy = np.meshgrid(x_grid, y_grid, sparse=True)
    h = land_func(xx,yy) # creates 20 x 20 meshgrid as longer range chemoreception search

    interest = {}

    for i in range(len(h)):
        for j in range(len(h[i])):
            if h[i][j]>=threshold:
                interest.update({h[i][j]:(x_grid[j],y_grid[i])}) # points above a threshold are added to dictionary
    sorted_list = sorted(interest.keys())

    angle_list = []

    for i in sorted_list[:2]:
        coord = interest.get(i) # returns two angles for the starting tendrils, using just one tendril at the start can result in it
                                # veering off course so this bolsters the chance of it finding an optima
        if (coord[0]-x)<0:
            angle_return = (180*2*(math.pi)/360) + np.arctan((coord[1]-y)/(coord[0]-x)) # using a quadrent system on the left two
                                                                                        # quadrents (where the change in X is negative)
                                                                                        # angle (degrees) is 180 + tan-1(delta_y/delta_x)
        if (coord[0]-x)>=0:
            angle_return = np.arctan((coord[1]-y)/(coord[0]-x)) # right two quadrents (where the change in X is positive) 
                                                                # angle is simply tan-1(delta_y/delta_x)
        
        angle_list.append(angle_return)

        
    if len(angle_list) < 1: # if no optima above threshold are found, slime climb will reach out in 3 arbitrary directions to explore
        angle_list = [random.randint(0,120),random.randint(120,240),random.randint(240,360)]

    return(angle_list)
chemotaxis(0,3,1,30,0.5,NK_Landscape)

[-0.7853981633974483, -0.7298996581517315]

In [16]:
# The angle range doesn't account the direction, in reality the slime mold zig zags as it explores its enviroment, this
# positive/negative function accounts for this

def positive_or_negative():
    return 1 if random.random() < 0.5 else -1

In [17]:
# Function simply graphs each 'wave/iteration of points as the tendrils spread out, for this reason it takes longer to run
# than the actual function but is very useful to visualise the slime mold's behaviour

def plot_drawer_2d(x,y,func,land_func):
    
    if str(land_func.__name__) == 'rastrigin' :

        lower,upper = -8,8
    else:
        lower,upper = -3,7

    fig = plt.figure(figsize=(20,15))
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(90, 0)

    x1 = np.linspace(lower,upper,100)
    y1 = np.linspace(lower,upper,100)
    xx, yy = np.meshgrid(x1, y1, sparse=False)
    z = land_func(xx, yy)
    ax.plot_surface(xx, yy, z, cmap='RdBu',alpha=0.4,zorder=0)
    fig.canvas.draw()

    ax.scatter(x[0][0],y[0][0],land_func(x[0][0],y[0][0]),color = 'k',s=100,alpha=1)

  

    for i in range(len(x)):
        clear_output()
        print('Plotter progress = ',int(i*100/len(x)),'%')

        for j in range(len(x[i])):
            h = land_func(x[i][j],y[i][j])
            ax.scatter(x[i][j],y[i][j],h,color = 'r',s=3,alpha=1)

In [18]:
def gify_2d(file_names,land_func):

    clear_output()
    print('Please wait source images being purged and gif is being compiled')
    
    images=[]    
    for filename in file_names:
        images.append(imageio.imread(filename))
    imageio.mimsave('Slime_mold_' + str(land_func.__name__) +  '.gif', images)

    
    for filename in file_names[:-1]:
        os.remove(filename)
        
    clear_output()

**Slime_Climb()**  
The tenrdil nodes' x,y,z coordinates are stored in a list of lists as well as the cumulative angle for from the predecessor node, the cumulative angle is vital in mapping the tendril direction over time as it's the basis of forward kinematics.

For our start we use the chemotaxis function to set our start direction angles, then we use our P_roll,dist_func and angle_spread functions over time to dictate how the tendrils behave.

In [19]:
def Slime_Climb(graph,end_time,seed,plot_func,land_func): 
    
    if str(land_func.__name__) == 'rastrigin' :
        lower,upper = -8,8
        
    else:
        lower,upper = -3,7
    
    dist = 0.5

    tendril1x  = [] # out list of lists for x,y,z coordinates and angles
    tendril1y  = []
    tendril1h = []
    tendril_h = []
    tendril_a = []

    branch_counter = 1 # we seed with just one branch, our spore

    tendril1x.append([seed[0]]) # Adds origin to pathway 
    tendril1y.append([seed[1]])

    tendril1h.append([land_func(seed[0],seed[1])]) # We need two previous refernce points for algorithm to function as it uses
    tendril1h.append([land_func(seed[0],seed[1])]) # previous two nodes for bifurication,angle and distance 'decisions' 

    tendril_a.append([0]) # The reference point for angular orientation is 0 degrees is a horizontal line starting at the origin
                          # travelling out to the right size of the origin (seed) 

    h_best = [land_func(seed[0],seed[1]),seed[0],seed[1]]

    for i in range(end_time): # For loop for our time increments

        loc_tend_list_x = []
        loc_tend_list_y = []
        loc_tend_list_a = []
        loc_tend_list_h = []

        for j in range(0,len(tendril1x[-1])): #  For loop for the previous timestep list of nodes

            prev_angle  = tendril_a[-1][j] # Loads the previous angle from the previous node to add to the new angle

            prev_height = tendril1h[-1][j] # From the previous node we find its height for comaprison with the node before that

            if prev_height > h_best[0]: # Check to see what the best heigh found is
                h_best = [prev_height,tendril1x[-1][j],tendril1y[-1][j]]
                
            if i == 0:
                start_angle_set = chemotaxis(5,seed[0],seed[1],end_time,dist,land_func) # Sets our starting angles from chemotaxis
                local_branch = int(len(start_angle_set))
                branch_switch = [1,1,1] # If we have a given angle say 50 degrees, we need to position each tendril at equidistant
                                        # points in the fork, the branch switch allows us to plot -25,25 and 0 for our 50 degrees 
                                        # giving a tri fork, if it is ony a bi fork or extension then the branch switch is indexed 
                                        # in a way that allows for this, for initial and forking, this is just the angle(s) returned
            else:                       # By the chemotaxis function
                local_branch = P_roll(prev_height,prev_prev_height,end_time,i) # For each of the previous nodes we roll to see
                if len(branch_switch) < 2:                                     # what the bifurication will be for this increment
                    branch_switch = 0.1
                else:
                    branch_switch = [-0.5,0.5,0] 
            
            for k in range(0,local_branch): # For loop for our new nodes for this increment

                if i == 0:
                    new_angle = start_angle_set[k]

                else :

                    low,high = angle_spread(prev_height,prev_prev_height,end_time,i)

                    new_angle = ((random.uniform(low,high))*(2*math.pi/360))*positive_or_negative()

                prev_prev_height = tendril1h[-2][int(k/local_branch)]

                dist = dist_func(prev_height,prev_prev_height)

                angle_loc = (branch_switch[k]*new_angle) + prev_angle

                x_val = dist*np.cos(angle_loc) + tendril1x[-1][j]
                y_val = dist*np.sin(angle_loc) + tendril1y[-1][j]

                h_new = land_func(x_val,y_val)

                loc_tend_list_h.append(h_new) # add our list of angles and coordinates into this increments set of entries
                loc_tend_list_x.append(x_val)
                loc_tend_list_y.append(y_val)

                loc_tend_list_a.append(angle_loc)

        tendril1x.append(loc_tend_list_x) # add list of this increments set of entires into list of lists
        tendril1y.append(loc_tend_list_y)
        tendril1h.append(loc_tend_list_h)
        tendril_a.append(loc_tend_list_a)

        branch_counter = len(tendril1x[-1])
        if graph == 1:
            clear_output()
            print('Algorithm progress = ',int(i*100/end_time), '%')

    if graph == 1:
        plot_func(tendril1x,tendril1y,land_func)
        
        clear_output()
    
        print('number of total number of branches on outer radius = ',branch_counter)

    return(h_best)

Slime_Climb(0,24,(3,7),gify_2d,NK_Landscape)

[12.124412766351274, -0.09071020945180347, 1.5624913027679677]

In [ ]:
def Slime_Climb(graph,end_time,seed,plot_func,land_func,agar): 
    
    if str(land_func.__name__) == 'bohav' :
        lower,upper = -100,100
        
    if str(land_func.__name__) == 'rastrigin' :
        lower,upper = -8,8
        
    else:
        lower,upper = -7,7
        

    if graph ==1:
    
        fig = plt.figure(figsize=(20,10))
        ax = fig.add_subplot(121, projection='3d')
        ax.view_init(35, 255)
        
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.view_init(90, 0)

        ax.set_xlim3d(lower, upper)
        ax.set_ylim3d(lower, upper)
        

        ax2.set_xlim3d(lower, upper)
        ax2.set_ylim3d(lower, upper)


        ax.scatter(seed[0],seed[1],land_func(seed[0],seed[1]),color = 'k',s=100,alpha=1)
        ax2.scatter(seed[0],seed[1],color = 'k',s=100,alpha=1)
        
        if str(land_func.__name__) == 'bohav' or str(land_func.__name__) == 'rastrigin' or str(land_func.__name__) == 'NK_Landscape':

            x1 = np.linspace(lower,upper,100)
            y1 = np.linspace(lower,upper,100)
            xx, yy = np.meshgrid(x1, y1, sparse=False)
            z = land_func(xx, yy)
            ax.plot_surface(xx, yy, z, cmap='RdBu',alpha=0.4,zorder=0)
            ax2.plot_surface(xx, yy, z, cmap='RdBu',alpha=0.4,zorder=0)

            fig.canvas.draw()

    
        c = ['r' , 'g' , 'b', 'y' , 'k']
        file_names = []

    dist = 0.5

    tendril1x  = [] # out list of lists for x,y,z coordinates and angles
    tendril1y  = []
    tendril1h = []
    tendril_h = []
    tendril_a = []

    branch_counter = 1 # we seed with just one branch, our spore

    tendril1x.append([seed[0]]) # Adds origin to pathway 
    tendril1y.append([seed[1]])

    tendril1h.append([land_func(seed[0],seed[1])]) # We need two previous refernce points for algorithm to function as it uses
    tendril1h.append([land_func(seed[0],seed[1])]) # previous two nodes for bifurication,angle and distance 'decisions' 

    tendril_a.append([0]) # The reference point for angular orientation is 0 degrees is a horizontal line starting at the origin
                          # travelling out to the right size of the origin (seed) 

    h_best = [land_func(seed[0],seed[1]),seed[0],seed[1]]

    for i in range(end_time): # Loop for our time increments

        loc_tend_list_x = []
        loc_tend_list_y = []
        loc_tend_list_a = []
        loc_tend_list_h = []

        for j in range(0,len(tendril1x[-1])): #  For loop for the previous timestep list of nodes

            prev_angle  = tendril_a[-1][j] # Loads the previous angle from the previous node to add to the new angle

            prev_height = tendril1h[-1][j] # From the previous node we find its height for comaprison with the node before that

            if prev_height > h_best[0]: # Check to see what the best heigh found is
                h_best = [prev_height,tendril1x[-1][j],tendril1y[-1][j]]
                
            if i == 0:
                start_angle_set = chemotaxis(5,seed[0],seed[1],end_time,dist,land_func) # Sets our starting angles from chemotaxis
                local_branch = int(len(start_angle_set))
                branch_switch = [1,1,1] # If we have a given angle say 50 degrees, we need to position each tendril at equidistant
                                        # points in the fork, the branch switch allows us to plot -25,25 and 0 for our 50 degrees 
                                        # giving a tri fork, if it is ony a bi fork or extension then the branch switch is indexed 
                                        # in a way that allows for this, for initial and forking, this is just the angle(s) returned
            else:                       # By the chemotaxis function
                local_branch = P_roll(prev_height,prev_prev_height,end_time,i) # For each of the previous nodes we roll to see
                if len(branch_switch) < 2:                                     # what the bifurication will be for this increment
                    branch_switch = 0.1
                else:
                    branch_switch = [-0.5,0.5,0] 
            
            for k in range(0,local_branch): # For loop for our new nodes for this increment

                if i == 0:
                    new_angle = start_angle_set[k]

                else :

                    low,high = angle_spread(prev_height,prev_prev_height,end_time,i)

                    new_angle = ((random.uniform(low,high))*(2*math.pi/360))*positive_or_negative()  
                    
                prev_prev_height = tendril1h[-2][int(k/local_branch)]

                dist = dist_func(prev_height,prev_prev_height)

                angle_loc = (branch_switch[k]*new_angle) + prev_angle

                x_val = dist*np.cos(angle_loc) + tendril1x[-1][j]
                y_val = dist*np.sin(angle_loc) + tendril1y[-1][j]

                h_new = land_func(x_val,y_val)
                
                if graph == 1:
                    colour = c[local_branch - 1]

                    ax.plot([x_val,tendril1x[-1][j]],[y_val,tendril1y[-1][j]],[h_new,prev_height],c=colour)
                    ax2.plot([x_val,tendril1x[-1][j]],[y_val,tendril1y[-1][j]],c=colour)
                
                loc_tend_list_h.append(h_new) # add our list of angles and coordinates into this increments set of entries
                loc_tend_list_x.append(x_val)
                loc_tend_list_y.append(y_val)

                loc_tend_list_a.append(angle_loc)
                
        if graph == 1:
                
            plt.tight_layout()
            
            if i == (end_time-1):
                print('True')
                file_names.append(str(land_func.__name__) + str(agar) + '.png')
                fig.savefig(str(land_func.__name__) + str(agar) + '.png')
                
            else:
                file_names.append(str(i) + '3oqChK0YHT' + '.png')
                fig.savefig(str(i) + '3oqChK0YHT' + '.png')
                

        tendril1x.append(loc_tend_list_x) # add list of this increments set of entires into list of lists
        tendril1y.append(loc_tend_list_y)
        tendril1h.append(loc_tend_list_h)
        tendril_a.append(loc_tend_list_a)

        branch_counter = len(tendril1x[-1])
        if graph == 1:
            clear_output()
            print('Algorithm progress = ',int(i*100/end_time), '%')

    if graph == 1:
        plot_func(file_names,land_func)
        
        clear_output()
    
    print('number of total number of branches on outer radius = ',branch_counter)

    return(h_best,tendril1x[-1],tendril1y[-1])

Slime_Climb(1,20,(-4,-4),gify_2d,rastrigin,1.5)

# Control Algorithms 

In [ ]:
# Basic Hill climber function to test against the Slime climb algorithm

def hill_climb(x_int,y_int,steps,mut,land_func):
    x = x_int
    y = y_int
    height = land_func(x,y)
    x_y_best = [x,y]
    height_best = height
    path_x = []
    path_y = []
    for i in range(0,steps):

        p = np.random.randint(0,100)   
        p_local = np.random.randint(0,100)       

        if p >= 50 : 
            y_alter = y + (random.uniform(-mut,mut)); x_alter = x
        if p < 50 : 
            x_alter = x + (random.uniform(-mut,mut)); y_alter = y
        height_alter = land_func(x_alter,y_alter)         

        if height_alter > height or p_local <10 :
            x = x_alter
            y = y_alter
            height = height_alter            

        if height > height_best: 
            height_best = height
            x_y_best[0] = x
            x_y_best[1] = y
        path_x.append(x)
        path_y.append(y)

    return(x_y_best[0],x_y_best[1], height_best)

hill_climb(0,1,100,0.5,NK_Landscape)

In [ ]:
# Basic Genetic algorithm with Mutation and tournament to test a population against the Slime climb algorithm

def mutate(coords,land_func):
    prev_fitness = land_func(coords[0],coords[1])

    new_coords = [0,0]
    new_coords[0] = coords[0] + random.uniform(-1,1)
    new_coords[1] = coords[1] + random.uniform(-1,1)

    new_fitness = land_func(new_coords[0],new_coords[1])

    if new_fitness >= prev_fitness:
        return(new_coords)

    else:
        return(coords)

def tournament(coords1,coords2,land_func):
    fit_1 = land_func(coords1[0],coords1[1])
    fit_2 = land_func(coords2[0],coords2[1])

    if fit_1 >= fit_2:
        return(1)
    if fit_2 >= fit_1:
        return(2)
    else:
        pass

def GA(lims,land_func):
    pop_int = {}
    for i in range(0,100):
        
        cords_new = [random.uniform(-3,7),random.uniform(-3,7)]
        perf = land_func(cords_new[0],cords_new[1])
        pop_int.update({perf:cords_new})

        for i in range(0,lims):

            best = sorted(pop_int.keys())[-1]

            pop = random.choice(list(pop_int.keys()))

            coordinates = pop_int.get(pop)
            del pop_int[pop]

            updated_coords = mutate(coordinates,land_func)
            updated_fitness = land_func(updated_coords[0],updated_coords[1])


            pop_int.update({updated_fitness:updated_coords})

            best = sorted(pop_int.keys())[-1]

            pop1 = random.choice(list(pop_int.keys()))
            pop2 = random.choice(list(pop_int.keys()))

            pop1_coords = pop_int.get(pop1)
            pop2_coords = pop_int.get(pop2)


            fight = tournament(pop1_coords,pop2_coords,land_func)

            if fight == 1:
                del pop_int[pop2]
                pop_int.update({pop1:pop1_coords})

            if fight == 2:
                del pop_int[pop1]
                pop_int.update({pop2:pop2_coords})

    return(best,pop_int.get(best))

GA(100,NK_Landscape)

# Test Functions

In [ ]:
# Timeit function finds the average length of time to complete a run of a given algorithm aswell as the average height achieved

def timeit(condition,increment,land_func):

    fp_avg = 0
    avg_time = 0
   
    start_cond_x = np.linspace(-3,7,4)
    start_cond_y = np.linspace(-3,7,4)
    
    for i in start_cond_x:
        for j in start_cond_y:

            if condition == 0:
                start=time.time()
                fp = hill_climb(i,j,increment,0.5,land_func)[2]

                end=time.time()
                avg_time += (end-start)
                fp_avg+=fp

            if condition == 1:
                start=time.time()
                fp = Slime_Climb(0,increment,(i, j),gify_2d,land_func)[0]
                end=time.time()           
                avg_time += (end-start)

                fp_avg+=fp

            if condition == 2:
                start=time.time()
                fp,coords = GA(increment,land_func)
                end=time.time()           
                avg_time += (end-start)
                fp_avg+=fp

    avg_time = avg_time/(16)
    fp_avg = fp_avg/(16)

    return(avg_time,fp_avg)

In [ ]:
# Use timeit function successively with all three algorithms to see time complexity and average height achieved (performance)

def time_performance_comparison(land_func):
    
    font_size(16) 
    
    fig = plt.figure(figsize = (30,10))

    ax = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)


    hill_hei= []
    slime_hei = []
    GA_hei = []

    time_s = []
    time_h = []
    time_GA =[]
    
    slime_scale = []
    hill_scale  = []
    GA_scale = []

    for i in range(1,25):

        time_slime,final_slime = timeit(1,int(i),land_func)
        time_hill,final_hill = timeit(0,int(i*1000),land_func)
        time_GA_A,final_GA = timeit(2,int(i*10),land_func)

        slime_scale.append(i)
        hill_scale.append(i*1000)
        GA_scale.append(i*5)

        hill_hei.append(final_hill)
        slime_hei.append(final_slime)
        GA_hei.append(final_GA)
        
        time_s.append(time_slime)
        time_h.append(time_hill)
        time_GA.append(time_GA_A)

        clear_output()
        print(i*100/25,'%')
        
    clear_output()
    
     
    ax.plot(slime_scale,slime_hei,c='r',label='Slime Climb Final Height Value')
    ax.plot(slime_scale,hill_hei,c='b',label='Hill Climb Final Height Value')
    ax.plot(slime_scale,GA_hei,c='g',label='GA Final Height Value')
    ax.set_title('Maximum height reached against number of iterations ' + str(land_func.__name__))
    ax.set_xlabel('Increment')
    ax.set_ylabel('Height reached')
    ax.legend(bbox_to_anchor=(1,-0.05))


    ax2.plot(slime_scale,time_s,c='r',label='Slime climb timespan')
    ax2.plot(slime_scale,time_h,c='b',label='Hill climb timespan')
    ax2.plot(slime_scale,time_GA,c='g',label='GA timespan')
    ax2.set_title('Wallclock time against number of iterations ' + str(land_func.__name__))
    ax2.set_xlabel('Increment')
    ax2.set_ylabel('Average time for a given run (s)')
    ax2.legend(bbox_to_anchor=(1,-0.05))

In [ ]:
# To visualise the difference in versatility between the Hill climber and Slime climb algorithm 
# we can generate a grid of different starting positions for a given landscape and see 
# how heigh each algorithm gets from said starting position 

def pcolor_performance(land_func):
    fig = plt.figure(figsize = (30,10))

    ax0 = fig.add_subplot(121)
    ax1 = fig.add_subplot(122)
    
    x_range = np.linspace(-8,8,25)
    y_range = np.linspace(-8,8,25)

    grid = np.zeros((len(y_range),len(x_range)))
    grid_h = np.zeros((len(y_range),len(x_range)))
    
    for x in range(len(x_range)):
        
        for y in range(len(y_range)):
            
            fp = 0
            fp_h = 0
            
            for i in range(0,5):
                fp += Slime_Climb(0,17,(x_range[x], y_range[y]),gify_2d,land_func)[0]
                fp_h += hill_climb(x_range[x], y_range[y],(17*1000),0.5,land_func)[2]
                
            fp   = fp/5
            fp_h = fp_h/5
            
            grid[y][x] = fp
            grid_h[y][x] = fp_h
            
        clear_output()
        print(x*100/len(x_range),'%')
        
    clear_output()   
    
    font_size(22)    
    
    a = ax0.pcolormesh(x_range,y_range,grid)
    fig.colorbar(a, ax=ax0)
    ax0.set_title('Average height reached for different starting coordinates (Slime Climb) \n' + str(land_func.__name__))
    
    b = ax1.pcolormesh(x_range,y_range,grid_h)
    fig.colorbar(b, ax=ax1)
    ax1.set_title('Average height reached for different starting coordinates (Hill Climb) \n' + str(land_func.__name__))

    fig.tight_layout()

# Results

In [ ]:
Slime_Climb(1,20,(0,0),gify_2d,rastrigin)

In [ ]:
Slime_Climb(1,22,(7,7),gify_2d,NK_Landscape)

In [ ]:
time_performance_comparison(rastrigin)

In [ ]:
time_performance_comparison(NK_Landscape)

In [ ]:
pcolor_performance(rastrigin)

In [ ]:
pcolor_performance(NK_Landscape)